# import, URL

In [1]:
import pandas as pd
import numpy as np
#pip install beautifulsoup4
from bs4 import BeautifulSoup
import requests
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response= requests.get(url)
result_page= BeautifulSoup(response.content, "html.parser")

# Create a table

In [2]:
table = result_page.find('table')
table_items = result_page.find_all('tr')
table_head = table_items[0].find_all("th")
table_rows = table_items[1:]
column_names = ["PostalCode", "Borough", "Neighborhood"]
toronto_neighborhoods = pd.DataFrame(columns = column_names)
toronto_neighborhoods

,PostalCode,Borough,Neighborhood


# Edit the table

In [3]:
for row in table_rows:
    
    row = row.find_all("td")
    PostalCode = row[0].get_text()
    Borough = row[1].get_text()
    Neighborhood = row[2].get_text().strip("\n")
    
    if Neighborhood == "Not assigned" and Borough != "Not assigned":
        Neighborhood = Borough
    
    if Borough == "Not assigned":
        continue
    else:
        if PostalCode in toronto_neighborhoods["PostalCode"].tolist():
            index = toronto_neighborhoods["PostalCode"].tolist().index(PostalCode)
            toronto_neighborhoods.loc[[index], ["Neighborhood"]] = toronto_neighborhoods.loc[[index], ["Neighborhood"]] + ", " + Neighborhood 
        else:
            toronto_neighborhoods = toronto_neighborhoods.append({
                "PostalCode": PostalCode,
                "Borough": Borough,
                "Neighborhood": Neighborhood
            }, ignore_index = True)

IndexError: list index out of range

# Part 1. Turn into a data frame

In [4]:
toronto_neighborhoods.shape
toronto_neighborhoods

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


# Part 2. Create another data frame

In [5]:
import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

ImportError: No module named 'geocoder'

In [6]:
geo = pd.read_csv('http://cocl.us/Geospatial_data')

geo.rename(columns= {"Postal Code": "PostalCode"}, inplace= True)
geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
df2= pd.merge(toronto_neighborhoods, geo, how= 'right', on= 'PostalCode')
df2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


# Explore and cluster neighboorhoods in Toronto

In [8]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.cm as cm
import matplotlib.colors as colors
import json

In [9]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

# All requested packages already installed.



In [10]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

NameError: name 'latitude' is not defined

In [11]:
df1 = df2.drop('Neighborhood', axis=1)
df1.tail()

,PostalCode,Borough,Latitude,Longitude
98,M8X,Etobicoke,43.653654,-79.506944
99,M4Y,Downtown Toronto,43.665860,-79.383160
100,M7Y,East Toronto,43.662744,-79.321558
101,M8Y,Etobicoke,43.636258,-79.498509
102,M8Z,Etobicoke,43.628841,-79.520999


In [12]:
dummy = pd.get_dummies(df1['Borough'])
df1 = pd.concat([df1,dummy],axis=1)
df1.drop('Borough', axis=1, inplace= True)
df1.head()

,PostalCode,Latitude,Longitude,Central Toronto,Downtown Toronto,East Toronto,East York,Etobicoke,Mississauga,North York,Queen's Park,Scarborough,West Toronto,York
0,M3A,43.753259,-79.329656,0,0,0,0,0,0,1,0,0,0,0
1,M4A,43.725882,-79.315572,0,0,0,0,0,0,1,0,0,0,0
2,M5A,43.654260,-79.360636,0,1,0,0,0,0,0,0,0,0,0
3,M6A,43.718518,-79.464763,0,0,0,0,0,0,1,0,0,0,0
4,M7A,43.662301,-79.389494,0,0,0,0,0,0,0,1,0,0,0


In [13]:
X = df1.iloc[:,1:]

In [14]:
num_clusters = 5

k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=15)
k_means.fit(X)
labels = k_means.labels_

print(labels)

[1 1 2 1 4 0 3 1 4 2 1 0 3 1 4 2 0 0 3 4 2 0 3 4 2 2 3 1 1 4 2 4 3 1 1 4 2
 4 3 1 1 4 2 4 3 1 1 4 2 1 1 3 1 1 4 1 0 1 3 1 1 4 4 0 0 3 1 4 4 4 0 3 1 4
 4 4 4 0 3 4 2 4 3 4 2 3 4 2 0 0 3 2 2 0 0 3 2 2 0 2 4 0 0]


In [15]:
df1["Labels"] = labels
df1.head(5)

,PostalCode,Latitude,Longitude,Central Toronto,Downtown Toronto,East Toronto,East York,Etobicoke,Mississauga,North York,Queen's Park,Scarborough,West Toronto,York,Labels
0,M3A,43.753259,-79.329656,0,0,0,0,0,0,1,0,0,0,0,1
1,M4A,43.725882,-79.315572,0,0,0,0,0,0,1,0,0,0,0,1
2,M5A,43.654260,-79.360636,0,1,0,0,0,0,0,0,0,0,0,2
3,M6A,43.718518,-79.464763,0,0,0,0,0,0,1,0,0,0,0,1
4,M7A,43.662301,-79.389494,0,0,0,0,0,0,0,1,0,0,0,4


In [16]:
df1.groupby('Labels').mean()

,Latitude,Longitude,Central Toronto,Downtown Toronto,East Toronto,East York,Etobicoke,Mississauga,North York,Queen's Park,Scarborough,West Toronto,York
Labels,,,,,,,,,,,,,
0,43.669088,-79.521650,0.000000,0.0,0.000000,0.000000,0.705882,0.000000,0.0,0.000000,0.0,0.000000,0.294118
1,43.750727,-79.429338,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.0,0.000000,0.000000
2,43.654169,-79.383665,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
3,43.766229,-79.249085,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.0,0.000000,0.000000
4,43.680804,-79.392377,0.333333,0.0,0.185185,0.185185,0.000000,0.037037,0.0,0.037037,0.0,0.222222,0.000000
